## ДЗ 3. 
Основное задание:
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.
​

Терминология, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.
​
Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.
​
Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.
​
Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.
​
Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.
​
Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.
​
Задание 6 (опциональное): сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.
​
Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

In [1]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd

import catboost as cb
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
warnings.simplefilter("ignore")
%matplotlib inline

In [41]:
train = pd.read_csv('assignment_2_train.csv')
test = pd.read_csv('assignment_2_test.csv')

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))
train.head(2)

train.shape = 180000 rows, 394 cols
test.shape = 100001 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
train.tail(n=5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
179995,3166995,0,3958217,39.00,W,1877,310.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179996,3166996,0,3958237,59.95,W,10075,514.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179997,3166997,0,3958241,34.00,W,6053,122.0,150.0,mastercard,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179998,3166998,0,3958260,59.00,W,7726,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179999,3166999,0,3958317,226.00,W,17480,528.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Видно, что TransactionID растет вместе с ID, так что можно сделать рабиение без перемешивания, и это будет соответствовать задаче по разбиению по TransactionID


**Задание 1:** сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [4]:
target = train['isFraud']
data = train.set_index('TransactionID')
data.drop(['isFraud'], axis=1, inplace=True)

In [5]:
numerical_ft = data.select_dtypes(include=[np.number])
print(f'Всего числовых признаков {numerical_ft.shape[1]}')
numerical_ft= numerical_ft.columns.tolist()

Всего числовых признаков 378


In [6]:
categorical_ft = data.select_dtypes(include=[np.object])
print(f'Всего категориальных признаков {categorical_ft.shape[1]}')
categorical_ft = categorical_ft.columns.tolist()

Всего категориальных признаков 14


In [7]:
for col in categorical_ft:
    data[col] = data[col].astype(str)

In [35]:
test_target = test['isFraud']
test_data = test.set_index('TransactionID')

for col in categorical_ft:
    test_data[col] = test_data[col].astype(str)
test_data.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
3287000,1,7415038,226.00,W,12473,555.0,150.0,visa,226.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287001,0,7415054,3072.00,W,15651,417.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287002,0,7415081,319.95,W,13844,583.0,150.0,visa,226.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287003,0,7415111,171.00,W,11556,309.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3287004,0,7415112,107.95,W,10985,555.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
x_train, x_valid, y_train, y_valid = train_test_split(
    data, target, train_size=0.7, shuffle=False, random_state=25,
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("test_data.shape = {} rows, {} cols".format(*test_data.shape))

x_train.shape = 125999 rows, 392 cols
x_valid.shape = 54001 rows, 392 cols
test_data.shape = 100001 rows, 393 cols


In [36]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_ft)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_ft)
test_pool = cb.Pool(test_data, test_target, cat_features=categorical_ft)

In [37]:
cb_params = {
    "n_estimators": 2000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 50,
    "verbose": 100,
    "learning_rate": 0.01,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42,
    "cat_features": categorical_ft
}

In [16]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(train_pool, eval_set=valid_pool)

0:	test: 0.6221256	best: 0.6221256 (0)	total: 701ms	remaining: 23m 20s
100:	test: 0.8464322	best: 0.8464322 (100)	total: 56s	remaining: 17m 33s
200:	test: 0.8589559	best: 0.8589559 (200)	total: 1m 56s	remaining: 17m 24s
300:	test: 0.8686370	best: 0.8686370 (300)	total: 3m 3s	remaining: 17m 16s
400:	test: 0.8745718	best: 0.8745828 (399)	total: 4m 7s	remaining: 16m 28s
500:	test: 0.8801433	best: 0.8801433 (500)	total: 5m 16s	remaining: 15m 45s
600:	test: 0.8851255	best: 0.8851255 (600)	total: 6m 25s	remaining: 14m 57s
700:	test: 0.8886836	best: 0.8886836 (700)	total: 7m 32s	remaining: 13m 58s
800:	test: 0.8911853	best: 0.8911853 (800)	total: 8m 36s	remaining: 12m 53s
900:	test: 0.8930228	best: 0.8930228 (900)	total: 9m 38s	remaining: 11m 45s
1000:	test: 0.8947860	best: 0.8947860 (1000)	total: 10m 43s	remaining: 10m 42s
1100:	test: 0.8956145	best: 0.8956150 (1099)	total: 11m 47s	remaining: 9m 37s
1200:	test: 0.8959463	best: 0.8959463 (1200)	total: 12m 54s	remaining: 8m 35s
1300:	test: 0.8

In [38]:
predictions_train = model.predict_proba(train_pool)
predictions_valid = model.predict_proba(valid_pool)
predictions_lb = model.predict_proba(test_pool)

In [39]:
print(f'train: {round(roc_auc_score(y_train, predictions_train[:, 1]), 4)}')
print(f'test: {round(roc_auc_score(y_valid, predictions_valid[:, 1]), 4)}')
print(f'l_board: {round(roc_auc_score(test_target, predictions_lb[:, 1]), 4)}')

train: 0.9029
test: 0.9016
l_board: 0.8602


**Вывод**
Модель ведет себя адекватно при сравнении трейна и валидации, но сравнение с лидербордом говорит о некотором переобучении, довольно существенное падение.

**Задание 2:** сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [43]:
x_train, x_valid, y_train, y_valid = train_test_split(
    data, target, train_size=0.7, shuffle=False, random_state=25,
)

x_valid, x_test, y_valid, y_test = train_test_split(
    x_valid, y_valid, train_size=0.7, shuffle=True, random_state=25
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 125999 rows, 392 cols
x_valid.shape = 37800 rows, 392 cols
x_test.shape = 16201 rows, 392 cols


In [44]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_ft)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_ft)
test_pool = cb.Pool(x_test, y_test, cat_features=categorical_ft)
lb_pool = cb.Pool(test_data, cat_features=categorical_ft)

In [ ]:
model = cb.CatBoostClassifier(**cb_params)
model.fit(train_pool, eval_set=valid_pool)

0:	test: 0.5598128	best: 0.5598128 (0)	total: 828ms	remaining: 27m 34s
100:	test: 0.8265603	best: 0.8265603 (100)	total: 58.6s	remaining: 18m 22s
200:	test: 0.8444204	best: 0.8444204 (200)	total: 1m 56s	remaining: 17m 26s
300:	test: 0.8509534	best: 0.8509534 (300)	total: 2m 57s	remaining: 16m 40s
400:	test: 0.8532893	best: 0.8533632 (399)	total: 4m 2s	remaining: 16m 5s
500:	test: 0.8557426	best: 0.8557430 (499)	total: 5m 5s	remaining: 15m 13s
600:	test: 0.8568165	best: 0.8568681 (572)	total: 6m 6s	remaining: 14m 14s
700:	test: 0.8581630	best: 0.8581702 (698)	total: 7m 8s	remaining: 13m 14s
800:	test: 0.8591618	best: 0.8591627 (794)	total: 8m 12s	remaining: 12m 17s
900:	test: 0.8611610	best: 0.8611615 (898)	total: 9m 21s	remaining: 11m 25s
1000:	test: 0.8625656	best: 0.8625656 (1000)	total: 10m 21s	remaining: 10m 20s


In [ ]:
predictions_train = model.predict_proba(train_pool)
predictions_valid = model.predict_proba(valid_pool)
predictions_test = model.predict_proba(test_pool)
predictions_lb = model.predict_proba(lb_pool)

In [ ]:
print(f'train: {round(roc_auc_score(y_train, predictions_train[:, 1]), 4)}')
print(f'valid: {round(roc_auc_score(y_valid, predictions_valid[:, 1]), 4)}')
print(f'test: {round(roc_auc_score(y_valid, predictions_valid[:, 1]), 4)}')
print(f'l_board: {round(roc_auc_score(test_target, predictions_lb[:, 1]), 4)}')

Значение на валидационной выборке и тесте стали хуже, на лидерборде почти не изменились. 

**Задание 3:** построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [48]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
   
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
   
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [49]:
np.random.seed(27)
scores = create_bootstrap_metrics(test_target, predictions_lb[:, 1], roc_auc_score)

calculate_confidence_interval(scores)

(0.8550486742571126, 0.8685616052281834)

**Вывод** 
* Значение метрики на лидерборде попадает в доверительный интервал, что показывает неплохое качество валидации. Валидация устойчивая. Ширина доверительного интервала допустима.

**Задание 4:** выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [50]:
x_adv = pd.concat([
    data, test_data], axis=0
)
y_adv = np.hstack((np.zeros(data.shape[0]), np.ones(test_data.shape[0])))
assert x_adv.shape[0] == y_adv.shape[0]

In [52]:
model = xgb.XGBClassifier(n_estimators=100)
model.fit(x_adv[numerical_ft], y_adv)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred_adv = model.predict_proba(x_adv[numerical_ft])
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 4))

In [ ]:
y_pred = model.predict_proba(x_train[numerical_ft])
y_pred

In [ ]:
pd.cut(
    y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

**Вывод**
* Данные на трейне и тесте совсем не совпадают, что подтверждает roc_auc_score = 1.0 Модель идеально отличает данные трейна и теста

**Задание 5:** сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [36]:
cv_data = cb.cv(
    cb.Pool(data, target, cat_features=categorical_features),
    cb_params,
)

0:	test: 0.6518597	best: 0.6518597 (0)	total: 895ms	remaining: 29m 49s
100:	test: 0.8631351	best: 0.8631351 (100)	total: 1m 14s	remaining: 23m 21s
200:	test: 0.8797378	best: 0.8797378 (200)	total: 2m 28s	remaining: 22m 7s
300:	test: 0.8874365	best: 0.8874365 (300)	total: 3m 39s	remaining: 20m 40s
400:	test: 0.8919986	best: 0.8919986 (400)	total: 4m 52s	remaining: 19m 25s
500:	test: 0.8934776	best: 0.8934776 (500)	total: 6m 2s	remaining: 18m 4s
600:	test: 0.8942905	best: 0.8942905 (600)	total: 7m 11s	remaining: 16m 44s
700:	test: 0.8953205	best: 0.8953206 (697)	total: 8m 19s	remaining: 15m 25s
800:	test: 0.8960420	best: 0.8960422 (799)	total: 9m 27s	remaining: 14m 9s
900:	test: 0.8970305	best: 0.8970305 (900)	total: 10m 34s	remaining: 12m 54s
1000:	test: 0.8991212	best: 0.8991246 (999)	total: 11m 45s	remaining: 11m 44s
1100:	test: 0.9008175	best: 0.9008175 (1100)	total: 12m 57s	remaining: 10m 35s
1200:	test: 0.9014755	best: 0.9014755 (1200)	total: 14m 7s	remaining: 9m 23s
1300:	test: 0.